In [15]:
#!/usr/bin/env python
import re, random, math, collections, itertools

PRINT_ERRORS=1

In [16]:
def readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia):

    #reading pre-labeled input and splitting into lines
    posSentences = open('rt-polarity.pos.txt', 'r', encoding="ISO-8859-1")
    posSentences = re.split(r'\n', posSentences.read())

    negSentences = open('rt-polarity.neg.txt', 'r', encoding="ISO-8859-1")
    negSentences = re.split(r'\n', negSentences.read())

    posSentencesNokia = open('nokia-pos.txt', 'r', encoding="ISO-8859-1")
    posSentencesNokia = re.split(r'\n', posSentencesNokia.read())

    negSentencesNokia = open('nokia-neg.txt', 'r', encoding="ISO-8859-1")
    negSentencesNokia = re.split(r'\n', negSentencesNokia.read())

    posDictionary = open('positive-words.txt', 'r', encoding="ISO-8859-1")
    posWordList = posDictionary.readlines()
    posWordList = [line.strip() for line in posWordList if not line.startswith(";") and not line == '\n']
    #posWordList = re.findall(r"[a-z\-]+", posDictionary.read())

    negDictionary = open('negative-words.txt', 'r', encoding="ISO-8859-1")
    negWordList = negDictionary.readlines()
    negWordList = [line.strip() for line in negWordList if not line.startswith(";") and not line == '\n']
    #negWordList = re.findall(r"[a-z\-]+", negDictionary.read())

    for i in posWordList:
        sentimentDictionary[i] = 1
    for i in negWordList:
        sentimentDictionary[i] = -1

    #create Training and Test Datsets:
    #We want to test on sentences we haven't trained on, to see how well the model generalses to previously unseen sentences

  #create 90-10 split of training and test data from movie reviews, with sentiment labels
    for i in posSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="positive"
        else:
            sentencesTrain[i]="positive"

    for i in negSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="negative"
        else:
            sentencesTrain[i]="negative"

    #create Nokia Datset:
    for i in posSentencesNokia:
            sentencesNokia[i]="positive"
    for i in negSentencesNokia:
            sentencesNokia[i]="negative"

In [17]:
#calculates p(W|Positive), p(W|Negative) and p(W) for all words in training data
def trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord):
    posFeatures = [] # [] initialises a list [array]
    negFeatures = []
    freqPositive = {} # {} initialises a dictionary [hash function]
    freqNegative = {}
    dictionary = {}
    posWordsTot = 0
    negWordsTot = 0
    allWordsTot = 0

    #iterate through each sentence/sentiment pair in the training data
    for sentence, sentiment in sentencesTrain.items():
        wordList = re.findall(r"[\w']+", sentence)

        #TO DO:
        #Populate bigramList (initialised below) by concatenating adjacent words in the sentence.
        #You might want to seperate the words by _ for readability, so bigrams such as:
        #You_might, might_want, want_to, to_seperate....

        bigramList=wordList.copy() #initialise bigramList
        for x in range(len(wordList)-1):
            bigramList.append(wordList[x]+"_" + wordList[x+1])
        
        trigramList=wordList.copy()
        for x in range(len(wordList)-2):
            trigramList.append(wordList[x]+"_"+ wordList[x+1]+"_"+ wordList[x+2])



        #-------------Finish populating bigramList ------------------#

        #TO DO: when you have populated bigramList, uncomment out the line below and , and comment out the unigram line to make use of bigramList rather than wordList

        for word in bigramList: #calculate over bigrams
        #for word in wordList: #calculate over unigrams
        #for word in trigramList:
            allWordsTot += 1 # keeps count of total words in dataset
            if not (word in dictionary):
                dictionary[word] = 1
            if sentiment=="positive" :
                posWordsTot += 1 # keeps count of total words in positive class

                #keep count of each word in positive context
                if not (word in freqPositive):
                    freqPositive[word] = 1
                else:
                    freqPositive[word] += 1
            else:
                negWordsTot+=1# keeps count of total words in negative class

                #keep count of each word in positive context
                if not (word in freqNegative):
                    freqNegative[word] = 1
                else:
                    freqNegative[word] += 1

    for word in dictionary:
        #do some smoothing so that minimum count of a word is 1
        if not (word in freqNegative):
            freqNegative[word] = 1
        if not (word in freqPositive):
            freqPositive[word] = 1

        # Calculate p(word|positive)
        pWordPos[word] = freqPositive[word] / float(posWordsTot)

        # Calculate p(word|negative)
        pWordNeg[word] = freqNegative[word] / float(negWordsTot)

        # Calculate p(word)
        pWord[word] = (freqPositive[word] + freqNegative[word]) / float(allWordsTot)

#---------------------------End Training ----------------------------------


In [18]:
#implement naive bayes algorithm
#INPUTS:
#  sentencesTest is a dictonary with sentences associated with sentiment
#  dataName is a string (used only for printing output)
#  pWordPos is dictionary storing p(word|positive) for each word
#     i.e., pWordPos["apple"] will return a real value for p("apple"|positive)
#  pWordNeg is dictionary storing p(word|negative) for each word
#  pWord is dictionary storing p(word)
#  pPos is a real number containing the fraction of positive reviews in the dataset
def testBayes(sentencesTest, dataName, pWordPos, pWordNeg, pWord,pPos):
    pNeg=1-pPos

    #These variables will store results (you do not need them)
    total=0
    correct=0
    totalpos=0
    totalpospred=0
    totalneg=0
    totalnegpred=0
    correctpos=0
    correctneg=0

    #for each sentence, sentiment pair in the dataset
    for sentence, sentiment in sentencesTest.items():
        wordList = re.findall(r"[\w']+", sentence)#collect all words


        #TO DO: Exactly what you did in the training function:
        #Populate bigramList by concatenating adjacent words in wordList.

        bigramList=wordList.copy() #initialise bigramList
        for x in range(len(wordList)-1):
            bigramList.append(wordList[x]+"_" + wordList[x+1])
        
        trigramList=wordList.copy()
        for x in range(len(wordList)-2):
            trigramList.append(wordList[x]+"_"+ wordList[x+1]+"_"+ wordList[x+2])

#------------------finished populating bigramList--------------
        pPosW=pPos
        pNegW=pNeg

        for word in bigramList: #calculate over bigrams
        #for word in wordList: #calculate over unigrams
        #for word in trigramList:
            if word in pWord:
                if pWord[word]>0.00000001:
                    pPosW *=pWordPos[word]
                    pNegW *=pWordNeg[word]

        prob=0;
        if pPosW+pNegW >0:
            prob=pPosW/float(pPosW+pNegW)
            #print('Probability  :', prob,'pPosW  :',pPosW, 'pNegW  :', pNegW)

        total+=1
        if sentiment=="positive":
            totalpos+=1
            if prob>0.5:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
                if PRINT_ERRORS:
                    print ("ERROR (pos classed as neg %0.2f):" %prob + sentence)
        else:
            totalneg+=1
            if prob<=0.5:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
                if PRINT_ERRORS:
                    print ("ERROR (neg classed as pos %0.2f):" %prob + sentence)
                    
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")

    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred)
    recall_neg=correctneg/float(totalneg)
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg);

    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")


In [19]:
# This is a simple classifier that uses a sentiment dictionary to classify
# a sentence. For each word in the sentence, if the word is in the positive
# dictionary, it adds 1, if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive",
# otherwise as "Negative"
def testDictionary(sentencesTest, dataName, sentimentDictionary, threshold, score_count=3):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    count=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        
        for word in Words:
            if word in sentimentDictionary:
                score+=sentimentDictionary[word]

        total+=1      
                
                
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1

    #print(count)
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")
    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred) if (totalnegpred) else 0
    recall_neg=correctneg/float(totalneg) #if (totalneg) else 0
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg) if (precision_neg+recall_neg) else 0;


    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")


In [29]:
# This is a simple classifier that uses a sentiment dictionary to classify
# a sentence. For each word in the sentence, if the word is in the positive
# dictionary, it adds 1, if it is in the negative dictionary, it subtracts 1.
# If the final score is above a threshold, it classifies as "Positive",
# otherwise as "Negative"
def testD(sentencesTest, dataName, sentimentDictionary, threshold, score_count=2):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    count=0
    lst = []
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        left_split = []
        not_list = []
        for word in Words:
            
            if 'not' in Words:
                for i in range(len(Words)-3):
                    if Words[i] == 'not':
                        not_list.append(Words[i+1])
                        #not_list.append(Words[i+2])
                        #not_list.append(Words[i+3])
                    else:
                        pass
                        #x = Words.index('not')
                        #y = Words[Words.index('not')+1]
                
                #print(y)
                if len(not_list)>0:
                    for z in not_list:
                        if z in sentimentDictionary:
                            if sentimentDictionary[z] == -1:
                                score += score_count
                            elif sentimentDictionary[z] == 1:
                                score -= score_count
                        else:
                            score +=0
                            
                
            elif 'but' in Words:
                for i in range(len(Words) - 1):
                    left_split.append(Words[i])
                    if Words[i + 1] == 'but':
                        break
            
                #print(left_split)
        
                if len(left_split) > 0:
                    for word in left_split:
                        if word in sentimentDictionary:
                            if sentimentDictionary[word] == -1:
                                score += score_count
                            elif sentimentDictionary[word] == 1:
                                score -= score_count
                            else:
                                score +=0
                            count+=1
                            #print(score)     
            elif word in sentimentDictionary:
                score+=sentimentDictionary[word]
                
        #print(sentence, sentiment)
        total+=1
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1

    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")
    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred) if (totalnegpred) else 0
    recall_neg=correctneg/float(totalneg) #if (totalneg) else 0
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg) if (precision_neg+recall_neg) else 0;


    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")
    
#testD(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, 0)
testD(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, 0)
#testD(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", sentimentDictionary, 0)

Films  (Test Data, Rule-Based)	 Accuracy (All)=0.63 (686/1087)

Films  (Test Data, Rule-Based)	 Precision (Pos)=0.60 (419/703)
Films  (Test Data, Rule-Based)	 Recall (Pos)=0.78 (419/536)
Films  (Test Data, Rule-Based)	 F-measure (Pos)=0.68
Films  (Test Data, Rule-Based)	 Precision (Neg)=0.70 (267/384)
Films  (Test Data, Rule-Based)	 Recall (Neg)=0.48 (267/551)
Films  (Test Data, Rule-Based)	 F-measure (Neg)=0.57



In [21]:
#Print out n most useful predictors
def mostUseful(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]<0.0000001:
            predictPower[word]=1000000000
        else:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])


    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    print ("NEGATIVE:")
    print (head)
    print ("\nPOSITIVE:")
    print (tail)


In [22]:
#---------- Main Script --------------------------


sentimentDictionary={} # {} initialises a dictionary [hash function]
sentencesTrain={}
sentencesTest={}
sentencesNokia={}

In [23]:
readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia)

pWordPos={} # p(W|Positive)
pWordNeg={} # p(W|Negative)
pWord={}    # p(W)


In [24]:
#build conditional probabilities using training data
trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord)

#run naive bayes classifier on datasets
#print ("Naive Bayes")
testBayes(sentencesTrain,  "Films (Train Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
#testBayes(sentencesTest,  "Films  (Test Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesNokia, "Nokia   (All Data,  Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.7)


ERROR (pos classed as neg 0.18):the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 
ERROR (pos classed as neg 0.09):effective but too-tepid biopic
ERROR (pos classed as neg 0.24):perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions . 
ERROR (pos classed as neg 0.19):ultimately , it ponders the reasons we need stories so much . 
ERROR (pos classed as neg 0.31):a screenplay more ingeniously constructed than " memento " 
ERROR (pos classed as neg 0.05):if this movie were a book , it would be a page-turner , you can't wait to see what happens next . 
ERROR (pos classed as neg 0.30):a few artsy flourishes aside , narc is as gritty as a movie gets these days . 
ERROR (pos classed as neg 0.28):a romantic comedy that operates by the rules of its own self-contained universe . 
ERROR (pos classed as neg 0.33):4 friend

ERROR (pos classed as neg 0.00):writer-director david jacobson and his star , jeremy renner , have made a remarkable film that explores the monster's psychology not in order to excuse him but rather to demonstrate that his pathology evolved from human impulses that grew hideously twisted . 
ERROR (pos classed as neg 0.04):while we no longer possess the lack-of-attention span that we did at seventeen , we had no trouble sitting for blade ii . 
ERROR (pos classed as neg 0.33):like a poor man's you can count on me
ERROR (pos classed as neg 0.01):my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more . 
ERROR (pos classed as neg 0.48):the more you think about the movie , the more you will probably like it . 
ERROR (pos classed as neg 0.00):there has been a string of ensemble cast romances recently . . . but peter mattei's love in the time of money sets itself apart by forming a

Films (Train Data, Naive Bayes)	 Recall (Pos)=0.92 (4390/4795)
Films (Train Data, Naive Bayes)	 F-measure (Pos)=0.93
Films (Train Data, Naive Bayes)	 Precision (Neg)=0.92 (4511/4916)
Films (Train Data, Naive Bayes)	 Recall (Neg)=0.94 (4511/4781)
Films (Train Data, Naive Bayes)	 F-measure (Neg)=0.93

ERROR (pos classed as neg 0.01):there is much which has been said in other reviews about the features of this phone , it is a great phone , mine worked without any problems right out of the box . 
ERROR (pos classed as neg 0.04):i have had the phone for 1 week , the signal quality has been great in the detroit area ( suburbs ) and in my recent road trip between detroit and northern kentucky ( cincinnati ) i experienced perfect signal and reception along i-75 , far superior to at &#38; t 's which does not work along several long stretches on that same route . 
ERROR (pos classed as neg 0.32):my favorite features , although there are many , are the speaker phone , the radio and the infrared .

In [25]:
#run sentiment dictionary based classifier on datasets
testDictionary(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, -4)
testDictionary(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, -4)
testDictionary(sentencesNokia, "Nokia   (All Data, Rule-Based)\t", sentimentDictionary, -3)


Films (Train Data, Rule-Based)	 Accuracy (All)=0.50 (4824/9576)

Films (Train Data, Rule-Based)	 Precision (Pos)=0.50 (4786/9529)
Films (Train Data, Rule-Based)	 Recall (Pos)=1.00 (4786/4795)
Films (Train Data, Rule-Based)	 F-measure (Pos)=0.67
Films (Train Data, Rule-Based)	 Precision (Neg)=0.81 (38/47)
Films (Train Data, Rule-Based)	 Recall (Neg)=0.01 (38/4781)
Films (Train Data, Rule-Based)	 F-measure (Neg)=0.02

Films  (Test Data, Rule-Based)	 Accuracy (All)=0.50 (540/1087)

Films  (Test Data, Rule-Based)	 Precision (Pos)=0.49 (535/1081)
Films  (Test Data, Rule-Based)	 Recall (Pos)=1.00 (535/536)
Films  (Test Data, Rule-Based)	 F-measure (Pos)=0.66
Films  (Test Data, Rule-Based)	 Precision (Neg)=0.83 (5/6)
Films  (Test Data, Rule-Based)	 Recall (Neg)=0.01 (5/551)
Films  (Test Data, Rule-Based)	 F-measure (Neg)=0.02

Nokia   (All Data, Rule-Based)	 Accuracy (All)=0.70 (186/266)

Nokia   (All Data, Rule-Based)	 Precision (Pos)=0.70 (186/266)
Nokia   (All Data, Rule-Based)	 Recall (Po

In [26]:
# print most useful words
mostUseful(pWordPos, pWordNeg, pWord, 50)

NEGATIVE:
['stupid', 'badly', 'routine', 'generic', 'unfunny', 'mediocre', 'ends_up', 'the_problem', 'mindless', '90_minutes', 'supposed_to', 'problem_with', 'poorly', 'boring', 'bears', 'it_feels', 'tiresome', 'annoying', 'shoot', 'disguise', 'unless', 'meandering', 'stale', 'little_to', 'offensive', 'pointless', 'waste_of', 'chan', 'sit_through', 'only_thing', 'apparently', 'bore', 'the_worst', 'junk', 'inept', 'disaster', 'attempt_at', 'it_were', 'animal', 'so_bad', 'amateurish', 'not_enough', 'harvard', 'to_sit', 'wasted', 'plotting', 'other_than', 'product', 'seagal', 'plodding']

POSITIVE:
['captures', 'tender', 'playful', 'iranian', 'detailed', 'tour', 'captivating', 'spare', 'sides', 'best_films', 'jealousy', 'wry', 'unfolds', 'your_heart', 'a_compelling', 'beauty', 'lively', 'polished', 'vividly', 'a_sweet', 'dazzling', 'wonderfully', 'love_and', 'portrait_of', 'an_engaging', 'gem', 'mesmerizing', 'wonderful', 'heartwarming', 'extraordinary', 'charming_and', 'intense', 'a_terr

In [27]:
def head_tail(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]<0.0000001:
            predictPower[word]=1000000000
        else:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])


    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    
    return head, tail

In [28]:
pcnt=0
ncnt=0
head, tail = head_tail(pWordPos, pWordNeg, pWord, 50)
print(' Negative Words in SentimentDictionary are')
print('')
for i in range(len(head)):
    if head[i] in sentimentDictionary:
        ncnt+=1
        print(head[i])
    else:
        pass
print(' ')
print(' ')
print(' Positive Words in SentimentDictionary are')
print('')
for i in range(len(tail)):
    if tail[i] in sentimentDictionary:
        pcnt+=1
        print(tail[i])
    else:
        pass

print('total positive words in sentimentDictionary are:',pcnt,' total useful words',len(tail))
print('total negative words in sentimentDictionary are:',ncnt,' total useful words',len(head))


 Negative Words in SentimentDictionary are

stupid
badly
mediocre
mindless
poorly
boring
tiresome
annoying
stale
offensive
pointless
bore
junk
inept
disaster
wasted
 
 
 Positive Words in SentimentDictionary are

tender
playful
captivating
jealousy
beauty
lively
polished
dazzling
wonderfully
gem
mesmerizing
wonderful
heartwarming
extraordinary
intense
warm
realistic
refreshing
flawed
inventive
engrossing
total positive words in sentimentDictionary are: 21  total useful words 50
total negative words in sentimentDictionary are: 16  total useful words 50
